# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2022-04-15,"Evidence, causes, and consequences of declinin...",Science,NaN,"Mason, R. E., J. M. Craine, N. K. Lany, M. Jon...",mason-et-al-2022,https://doi.org/10.1126/science.abh3767
1,2022-04-04,Biodiversity effects of food system sustainabi...,PNAS,Featured in [Anthropocene Magazine](https://ww...,"Read, Q. D., K. L. Hondula, and M. K. Muth. 20...",read-et-al-2022-pnas,https://doi.org/10.1073/pnas.2113884119
2,2022-03-12,Identification of quantitative trait loci asso...,Plant Disease,NaN,"Sthapit Kandel, J., G. V. Sandoya Miranda, W. ...",sthapit-kandel-et-al-2022,https://doi.org/10.1094/PDIS-09-21-2087-RE
3,2022-01-03,Plant removal across an elevational gradient m...,Ecology,NaN,"Rewcastle, K. E., J. A. Henning, Q. D. Read, R...",rewcastle-et-al-2022,https://doi.org/10.1002/ecy.3546
4,2021-06-21,Linking terrestrial and aquatic biodiversity t...,Frontiers in Environmental Science,Open access!,"Dahlin, K. M., P. L. Zarnetske, Q. D. Read, L....",dahlin-et-al-2021,https://www.frontiersin.org/articles/10.3389/f...
5,2021-04-20,Climate and multiple dimensions of plant diver...,Ecosphere,Open access!,"Prager, C. M., X. Jing, J. A. Henning, Q. D. R...",prager-et-al-2021,https://dx.doi.org/10.1002/ecs2.3472
6,2021-04-01,Reducing water scarcity by reducing food loss ...,Frontiers in Sustainable Food Systems,Open access!,"Marston, L.T., Read, Q.D., Brown, S.P. & Muth,...",marston-et-al-2021,NaN
7,2021-02-02,Cost-effectiveness of four food waste interven...,"Resources, Conservation & Recycling",NaN,"Read, Q.D. and M. K. Muth. 2021. Cost-effectiv...",read-and-muth-2021,https://www.sciencedirect.com/science/article/...
8,2021-01-20,Nitrogen and the food system,One Earth,NaN,"Metson, G.S., A. Chaudhary, X. Zhang, B. Hoult...",metson-et-al-2021-oneearth,http://www.sciencedirect.com/science/article/p...
9,2020-06-23,Life history scaling and the division of energ...,bioRxiv (preprint),Open access!,"Grady, J. M., Q. D. Read, S. Record, N. Rüger,...",grady-et-al-2020-biorxiv,https://www.biorxiv.org/content/10.1101/2020.0...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [4]:
# Some test code to see what the filename would be
# Looks like excel messed it up.
str(publications['pub_date'])

'0     2022-04-15\n1     2022-04-04\n2     2022-03-12\n3     2022-01-03\n4     2021-06-21\n5     2021-04-20\n6     2021-04-01\n7     2021-02-02\n8     2021-01-20\n9     2020-06-23\n10    2020-06-01\n11    2020-01-24\n12    2020-01-14\n13    2019-07-16\n14    2019-02-27\n15    2019-03-12\n16    2018-10-20\n17    2018-10-01\n18    2018-01-24\n19    2018-03-06\n20    2018-05-01\n21    2017-09-01\n22    2017-07-01\n23    2017-12-29\n24    2016-07-01\n25    2016-07-01\n26    2016-08-01\n27    2015-09-01\n28    2014-02-01\n29    2013-12-07\n30    2013-11-12\n31    2012-02-29\nName: pub_date, dtype: object'

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = str(item.pub_date)[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    ##md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [8]:
!ls ../_publications/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!cat ../_publications/2020-01-14-read-et-al-2020-sci-tot-env.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
